## Create Training Set

In [18]:
!python3 ../src/coco_trainer.py train --dataset=../datasets/ikea_drill_screw --weights=coco

Using TensorFlow backend.

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE            

In [19]:
!aws s3 sync ../datasets s3://dkitdatasets

upload: ../datasets/.DS_Store to s3://dkitdatasets/.DS_Store                       
upload: ../datasets/ikea_drill_screw/input/.DS_Store to s3://dkitdatasets/ikea_drill_screw/input/.DS_Store
upload: ../datasets/ikea_drill_screw/input/foregrounds/.DS_Store to s3://dkitdatasets/ikea_drill_screw/input/foregrounds/.DS_Store
upload: ../datasets/ikea_drill_screw/input/foregrounds/instruments/.DS_Store to s3://dkitdatasets/ikea_drill_screw/input/foregrounds/instruments/.DS_Store
upload: ../datasets/ikea_drill_screw/input/foregrounds/instruments/screw/.DS_Store to s3://dkitdatasets/ikea_drill_screw/input/foregrounds/instruments/screw/.DS_Store
upload: ../datasets/ikea_drill_screw/train/.DS_Store to s3://dkitdatasets/ikea_drill_screw/train/.DS_Store
upload: ../datasets/ikea_drill_screw/.DS_Store to s3://dkitdatasets/ikea_drill_screw/.DS_Store
upload: ../datasets/ikea_drill_screw/input/foregrounds/instruments/screw/IMG_0177.png to s3://dkitdatasets/ikea_drill_screw/input/foregrounds/instruments/

upload: ../datasets/ikea_drill_screw/train/images/00000051.jpg to s3://dkitdatasets/ikea_drill_screw/train/images/00000051.jpg
upload: ../datasets/ikea_drill_screw/train/images/00000049.jpg to s3://dkitdatasets/ikea_drill_screw/train/images/00000049.jpg
upload: ../datasets/ikea_drill_screw/train/images/00000060.jpg to s3://dkitdatasets/ikea_drill_screw/train/images/00000060.jpg
upload: ../datasets/ikea_drill_screw/train/images/00000063.jpg to s3://dkitdatasets/ikea_drill_screw/train/images/00000063.jpg
upload: ../datasets/ikea_drill_screw/train/images/00000066.jpg to s3://dkitdatasets/ikea_drill_screw/train/images/00000066.jpg
upload: ../datasets/ikea_drill_screw/train/images/00000067.jpg to s3://dkitdatasets/ikea_drill_screw/train/images/00000067.jpg
upload: ../datasets/ikea_drill_screw/train/images/00000064.jpg to s3://dkitdatasets/ikea_drill_screw/train/images/00000064.jpg
upload: ../datasets/ikea_drill_screw/train/images/00000068.jpg to s3://dkitdatasets/ikea_drill_screw/train/imag

upload: ../datasets/ikea_drill_screw/train/masks/00000007.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000007.png
upload: ../datasets/ikea_drill_screw/train/masks/00000004.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000004.png
upload: ../datasets/ikea_drill_screw/train/masks/00000005.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000005.png
upload: ../datasets/ikea_drill_screw/train/masks/00000008.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000008.png
upload: ../datasets/ikea_drill_screw/train/masks/00000006.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000006.png
upload: ../datasets/ikea_drill_screw/train/masks/00000003.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000003.png
upload: ../datasets/ikea_drill_screw/train/masks/00000009.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000009.png
upload: ../datasets/ikea_drill_screw/train/masks/00000010.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000010.png


upload: ../datasets/ikea_drill_screw/train/masks/00000072.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000072.png
upload: ../datasets/ikea_drill_screw/train/masks/00000073.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000073.png
upload: ../datasets/ikea_drill_screw/train/masks/00000075.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000075.png
upload: ../datasets/ikea_drill_screw/train/masks/00000076.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000076.png
upload: ../datasets/ikea_drill_screw/train/masks/00000078.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000078.png
upload: ../datasets/ikea_drill_screw/train/masks/00000074.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000074.png
upload: ../datasets/ikea_drill_screw/train/masks/00000079.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000079.png
upload: ../datasets/ikea_drill_screw/train/masks/00000077.png to s3://dkitdatasets/ikea_drill_screw/train/masks/00000077.png


upload: ../datasets/ikea_drill_screw/val/images/00000019.jpg to s3://dkitdatasets/ikea_drill_screw/val/images/00000019.jpg
upload: ../datasets/ikea_drill_screw/val/images/00000004.jpg to s3://dkitdatasets/ikea_drill_screw/val/images/00000004.jpg
upload: ../datasets/ikea_drill_screw/val/images/00000022.jpg to s3://dkitdatasets/ikea_drill_screw/val/images/00000022.jpg
upload: ../datasets/ikea_drill_screw/val/images/00000009.jpg to s3://dkitdatasets/ikea_drill_screw/val/images/00000009.jpg
upload: ../datasets/ikea_drill_screw/val/images/00000023.jpg to s3://dkitdatasets/ikea_drill_screw/val/images/00000023.jpg
upload: ../datasets/ikea_drill_screw/val/images/00000024.jpg to s3://dkitdatasets/ikea_drill_screw/val/images/00000024.jpg
upload: ../datasets/ikea_drill_screw/val/images/00000029.jpg to s3://dkitdatasets/ikea_drill_screw/val/images/00000029.jpg
upload: ../datasets/ikea_drill_screw/val/masks/00000001.png to s3://dkitdatasets/ikea_drill_screw/val/masks/00000001.png
upload: ../dataset

## Create Validation Set

In [9]:
INPUT_PATH = '../datasets/ikea_drill_screw/input'
VAL_PATH = '../datasets/ikea_drill_screw/val'
VAL_COUNT = 30
IMAGE_SIZE = 1024
MASK_DEF_PATH = VAL_PATH + '/mask_definitions.json'
DI_PATH = INPUT_PATH + '/dataset_info.json'
#Generate the images
!python3 ../src/image_composition.py --input_dir {INPUT_PATH} --output_dir {VAL_PATH} --count {VAL_COUNT} --width {IMAGE_SIZE} --height {IMAGE_SIZE} --silent
#Generate the annotaions json
!python3 ../src/coco_json_utils.py -md {MASK_DEF_PATH} -di {DI_PATH}






../src/image_composition.py:197: UserWarning: file found in foregrounds directory (expected super-category directories), ignoring: ../datasets/ikea_drill_screw/input/foregrounds/.DS_Store
  warnings.warn(f'file found in foregrounds directory (expected super-category directories), ignoring: {super_category_dir}')
../src/image_composition.py:203: UserWarning: file found in super category directory (expected category directories), ignoring: ../datasets/ikea_drill_screw/input/foregrounds/instruments/.DS_Store
  warnings.warn(f'file found in super category directory (expected category directories), ignoring: {category_dir}')
../src/image_composition.py:212: UserWarning: foreground must be a .png file, skipping: ../datasets/ikea_drill_screw/input/foregrounds/instruments/screw/.DS_Store
  warnings.warn(f'foreground must be a .png file, skipping: {str(image_file)}')
Generating 30 images with masks...
100%|███████████████████████████████████████████| 30/30 [00:08<00:00,  3.70it/s]
Image composi